# Naive Bayes

## About the model

Naive Bayes is a technique used for classification that has been studied since the 1950s. It was first used for text categorization, predicting whether a text is of one type or another, spam or not spam, English or German, for instance. It is still used to this day as a benchmark for this class of tasks. Of course, it can be used for any classification task.

This is one of the first papers introducing Bayesian techniques, including Naive Bayes, if you're curious: https://www.cs.utexas.edu/~jsinapov/teaching/cs378/readings/W2/Minsky60steps.pdf

## Pros

- Highly scalable
- Easy to implement
- Doesn't require a lot of data
- Easily handles missing data and noise

## Cons

- Bad estimators of class probabilities
- Assumptions made often do not hold
- Simplistic

# Introduction

Naive Bayes classifiers are generative models. This means that they will try to learn how instances of a certain class are generated by looking at what they inherently have in common.

Let's illustrate this with an example. Let's say that we are given a dataset containing information about weight, height, and sex of people.

By looking at this data, we can infer that males seem to be taller and heavier than females. Therefore, if we had to generate new male data points, we would be more likely to generate people measuring 1m80 and weighting 85kg rather than people measuring 1m60 and weighting 60kg.

Therefore, if we encounter a person measuring 1m92 and weighting 102kg, we would classify them as males because we would be more likely to generate such a person as a male rather than a female.


## Bayes' Theorem

To represent this concept mathematically, we make use of Bayes' Theorem. Which is represented by the following formula:

$$P(Class \mid Features) = \frac{P(Class) P(Features \mid Class)}{P(Features)}$$

The theorem gives us a way to calculate the probability a class (male or female) given some features (height and weight). Therefore, to make a prediction, we will need to calculate the probability for both classes and pick the class with the highest one.

If we are trying to classify a person measuring 1m80 and weighting 80kg, Bayes' Formula takes the following form:

$$P(Gender \mid Height=180, Weight = 60) = \frac{P(Gender) \space P(Height=180, Weight = 60 \mid Gender)}{P(Height=180, Weight = 60)}$$

Which can be read in statistical term as the following:

$$Posterior = \frac{Prior \times Likelihood}{Evidence}$$

>### Prior

>The prior represents the belief that an instance is of a certain class "prior" to knowing its features. This is equivalent >to guessing whether someone is male or female without knowing anything about them.

>### Likelihood

>The likelihood represents how "likely" certain features are to appear given a class. For instance, it is more "likely" that a man is 1m90 rather than 1m50.

>### Evidence

>The evidence represents the belief that the features, which we use as "evidence" to infer the class, are correct. For instance, using the fact that a man is 1m80 and 80kg as evidence is stronger than using the fact that a male is 2m15 and 45kg, as it is much less probable to be true.

>### Posterior

>The posterior probability represents the belief that an event will occur "after" taking the features into account. For instance, how likely it is that someone is male, "after" we've considered that they measure 1m80 and weight 80kg.


To simplify the formula, we can ignore the evidence as it is independent of the class. This means that we are now calculating the **joint probability**, which is proportional to the **posterior probability**. We can also make use of the strong assumption of independence, which is why these classifiers are called "Naïve". What does this mean in practice? It means that our formula now looks like this:

$$P(Gender \mid Height=180, Weight = 60) \propto P(Gender) \space P(Height=180 \mid Gender) \space P(Weight = 60 \mid Gender)$$

The strong assumption of independence simplified the likelihood by assuming that gender and height are independent of one another. It means that now, to calculate the posterior, and ultimately being able to classify an instance, we need to calculate these probabilities.

## Implementation from "skratch"

Here, we will show how to implement a Naive Bayes classifier from scratch. We first start with a `predict` function. This function will compute the joint probability for each class as defined in `_predict_joint_proba`, and return the class with the highest joint probability.

As described above, the joint probability is defined as $Prior \times Likelihood$.

In [45]:
import numpy as np


class NBClassifier:

    def predict(self, X, y=None):

        joint_probas = self._predict_joint_proba(X)
        indices = np.argmax(joint_probas, axis=1)

        return self.classes_[indices]

    def _predict_joint_proba(self, X, y=None):

        return np.array([[self._get_prior(c) * self._get_likelihood(sample, c) for c in self.classes_]
                         for sample in X])

Since the joint probability is proportional but doesn't equal the exact probability, we can also have a function `predict_proba` computing the probabilities of each instance being of each class. This means that we divide the joint probability by the evidence.

In [46]:
    def predict_proba(self, X, y=None):

        joint_probas = self._predict_joint_proba(X, y)
        evidence = np.array([[self._get_evidence(x)] for x in X])

        return joint_probas / evidence

In order to fit the model, we need to learn how to compute the priors, the likelihood, and the evidence. And therefore, we also need to update them if we want to train on more data.

In [47]:
    def fit(self, X, y):

        self.priors_ = self._fit_prior(y)
        self.likelihood_ = self._fit_likelihood(X, y)
        self.evidence_ = self._fit_evidence(X)

        return self
    
    def update(self, X, y):

        self.priors_ = self._update_priors(y)
        self.likelihood_ = self._update_likelihood(X, y)
        self.evidence_ = self._update_evidence(X)

        return self

### Computing the probabilities

Now that we have shown the mechanics of fitting and predicting, we need to show how to deal with computing the required probabilities. Fitting and computing the priors is rather straightforward as it all relies on the frequencies of the classes in the training data.

In [48]:
    def _fit_prior(self, y):

        self.classes_, self.priors_ = np.unique(y, return_counts=True)

        return self.priors_

    def _update_priors(self, y):

        self.classes_, counts = np.unique(y, return_counts=True)
        self.priors_ += counts

        return self.priors_

    def _get_prior(self, c):

        return self.priors_[c] / np.sum(self.priors_)

In order to compute the evidence and likelihood however, one must assume the probability distribution that the features follow. And there are plenty of them available. This is why we often refer to Naïve Bayes classifiers, plural, one for each distribution.

There are three main types of Naïve Bayes classifiers:

### Gaussian Naive Bayes

Gaussian Naive Bayes assumes that the features follow a normal distribution. It is a useful model when dealing with continuous data. It is often considered as the "default" Naïve Bayes classifier.

In [49]:
import sys
sys.path.append("D:/source/skratch/source")

import numpy as np

from supervised.naive_bayes.nb_classifier import NBClassifier

EPSILON = 1E-16  # offset to avoid "divide by zero" errors


class GaussianNB(NBClassifier):

    def _pdf(self, x, mean, std):

        num = np.exp(-((x - mean)**2) / (EPSILON + 2 * std**2))
        den = np.sqrt(2 * np.pi * std**2) + EPSILON

        return num / den

Fitting the likelihood and the evidence comes down to estimating both parameters of the normal distribution, namely the mean and the standard deviation, for each feature. Fitting the likelihood is the same as fitting the evidence for each class.

In [50]:
    def _fit_evidence(self, X):

        feature_probas = []

        for feature in X.T:
            feature_probas.append(dict(mean=np.mean(feature),
                                       n=len(feature),
                                       std=np.std(feature, ddof=1)))

        return np.array(feature_probas)

    def _fit_likelihood(self, X, y):

        likelihood_ = []

        for c in self.classes_:
            samples = X[y == c]

            likelihood_.append(self._fit_evidence(samples))

        return np.array(likelihood_)

If we are given more data and we'd like to update the likelihood and evidence, we will need to update the mean and standard deviation. Luckily, we can do this without having to compute the mean and standard deviation from scratch. We can use the incremental mean and standard deviation formulae. These require that we keep track of how many instances the model was trained on, which is why we keep track of `n` as well.

In [51]:
    def _update_evidence(self, X):

        for i, feature in enumerate(X.T):
            self.evidence_[i] = self._update_mean_std_n(feature, self.evidence_[i])

        return self.evidence_

    def _update_likelihood(self, X, y):

        for c in self.classes_:
            samples = X[y == c]

            for i, feature in enumerate(samples.T):
                self.likelihood_[c][i] = self._update_mean_std_n(feature, self.likelihood_[c][i])

        return self.likelihood_

    def _update_mean_std_n(self, feature, mean_std_n):

        old_m = mean_std_n["mean"]
        old_std = mean_std_n["std"]
        old_n = mean_std_n["n"]

        n = old_n + len(feature)

        m = (old_m * old_n + np.mean(feature) * n) / (old_n + n)

        s = np.sqrt((old_n * (old_std**2 + (old_m - m)**2)
                     + len(feature) * (np.var(feature)
                                       + (np.mean(feature) - m)**2)
                     ) / (old_n + len(feature)))

        return dict(mean=m, std=std, n=n)

Finally, calculating the likelihood and evidence comes down to computing each feature's probability density function and multiply them together.

In [52]:
    def _get_evidence(self, sample):

        evidence = 1.0

        for i, feature in enumerate(sample):

            mean = self.evidence_[i]["mean"]
            std = self.evidence_[i]["std"]

            evidence *= self._pdf(feature, mean, std)

        return evidence

    def _get_likelihood(self, sample, c):

        likelihood = 1.0

        for i, feature in enumerate(sample):

            mean = self.likelihood_[c][i]["mean"]
            std = self.likelihood_[c][i]["std"]

            likelihood *= self._pdf(feature, mean, std)

        return likelihood

### Bernoulli Naive Bayes

Bernoulli Naive Bayes assumes that the features follow a Bernoulli distribution. Therefore, it also assumes that the features are binary. This classifier is useful as it takes the presence as well as the absence of features into account.

In [53]:
import numpy as np

from supervised.naive_bayes.nb_classifier import NBClassifier


class BernoulliNB(NBClassifier):

    def _pdf(self, x, p):

        return (1.0 - x) * (1.0 - p) + x * p

Fitting the evidence requires us to keep track of how many times a feature was positive, and how many times the feature occurred in total. Fitting the likelihood means that we need to calculate the evidence for each class.

In [54]:
    def _fit_evidence(self, X):

        feature_probas = [dict(count=np.sum(feature), n=len(feature)) for feature in X.T]

        return np.array(feature_probas)

    def _fit_likelihood(self, X, y):

        likelihood_ = []

        for c in self.classes_:
            samples = X[y == c]

            likelihood_.append(self._fit_evidence(samples))

        return likelihood_

Updating therefore only requires us to update these counts.

In [55]:
    def _update_evidence(self, X):

        for i, feature in enumerate(X.T):

            self.evidence[i]["count"] += np.sum(feature)
            self.evidence[i]["n"] += len(feature)

        return self.evidence_

    def _update_likelihood(self, X, y):

        for i, c in enumerate(self.classes_):
            samples = X[y == c]

            for i, feature in enumerate(samples.T):

                self.likelihood_[c][i]["count"] += np.sum(feature)
                self.likelihood_[c][i]["n"] += len(feature)

        return self.likelihood_

Finally, calculating the likelihood and evidence comes down to computing each feature's probability density function and multiply them together.

In [56]:
    def _get_evidence(self, sample):

        evidence = 1.0

        for i, feature in enumerate(sample):

            count = self.evidence_[i]["count"]
            n = self.evidence_[i]["n"]

            evidence *= self._pdf(x=feature, p=count / n)

        return evidence

    def _get_likelihood(self, sample, c):

        likelihood = 1.0

        for i, feature in enumerate(sample):

            count = self.likelihood_[c][i]["count"]
            n = self.likelihood_[c][i]["n"]

            likelihood *= self._pdf(x=feature, p=count / n)

        return likelihood

### Multinomial Naive Bayes

Bernoulli Naive Bayes assumes that the features follow a Multinomial distribution. It is quite useful to classify documents as the frequencies of certain words can affect the type of the document.

A good practice when dealing with categorical data is to use additive smoothing, also called Laplace smoothing. This is why we pass the `alpha` argument in the constructor.

In [57]:
from math import factorial as fact
from collections import Counter

import scipy.stats as ss
import numpy as np

from supervised.naive_bayes.nb_classifier import NBClassifier


class MultinomialNB(NBClassifier):

    def __init__(self, alpha=1.0):

        super().__init__()
        self.alpha = alpha

    def _pdf(self, x, p):

        f = fact(np.sum(x))

        for P, X in zip(p, x):
            f *= (P**X) / fact(X)

        return f

Fitting the evidence comes to down to counting how many times each feature occured in the total, and the likelihood is similar to computing the evidence for each class.

In [58]:
    def _fit_evidence(self, X):

        evidence_ = np.sum(X, axis=0)

        return evidence_

    def _fit_likelihood(self, X, y):

        likelihood_ = []

        for c in self.classes_:
            samples = X[y == c]

            likelihood_.append(self._fit_evidence(samples))

        return likelihood_

Updating then requires us to update these counts.

In [59]:
    def _update_evidence(self, X):

        self.evidence_ += np.sum(X, axis=0)

        return self.evidence_

    def _update_likelihood(self, X, y):

        for i, c in enumerate(self.classes_):
            samples = X[y == c]

            self.likelihood_[i] += np.sum(samples, axis=0)

        return likelihood_

Finally, calculating the likelihood and evidence comes down to computing each feature's probability density function and multiply them together.

In [60]:
    def _get_evidence(self, sample):

        p = []

        for i, feature in enumerate(sample):

            x = self.evidence_[i]
            N = np.sum(self.evidence_)
            d = len(sample)
            a = self.alpha

            prob = (x + a) / (N + (a * d))

            p.append(prob)

        return self._pdf(sample, p)

    def _get_likelihood(self, sample, c):

        p = []

        for i, feature in enumerate(sample):

            x = self.likelihood_[c][i]
            N = np.sum(self.likelihood_[c])
            d = len(sample)
            a = self.alpha

            prob = (x + a) / (N + (a * d))

            p.append(prob)

        return self._pdf(sample, p)

## FAQ

### Why is it called Naïve?

Makes use of the strong assumption of independence

### What probability distribution can be used?

The algorithm gives no guarantee to find the best clusters every time. Indeed, based on the first random clusters, the algorithm might converge to a different solution. This is why, K-Means is typically run multiple times in order to increase the chances to not get stuck with a bad solution.

### Can different features have different distributions?

Yes

### Is multinomial naive bayes with binary features the same as bernoulli naive bayes?

No

## Useful resources

Wikipedia:

   

Tutorials:  


Implementations:


Videos:

